In [ ]:
# !pip install googlemaps 
import os
import re
import time
import json
import pathlib
import googlemaps
import pandas as pd
import folium

API_KEY = ''
maps_client = googlemaps.Client(key=API_KEY)

# Passo 01 - Informações complementares das residências

In [12]:
def get_place_id(query, maps_client):
  """
  Busca o local usando Place Search e retorna o place_id do primeiro resultado.
  """
  try:
    location_search = maps_client.places(query=query, language='pt-BR')

    if location_search['results']:
        place_id = location_search['results'][0].get('place_id')
        return place_id

    return None

  except (googlemaps.exceptions.ApiError, googlemaps.exceptions.Timeout) as e:
    print(f"Erro na busca de Place ID para '{query}': {e}")
    return None

In [13]:
def get_place_details(place_id, query_original, maps_client):
  """
  Busca os detalhes complementares de um local usando o Place Details.
  """
  if not place_id:
    return None

  fields = [
    'name', 'place_id', 'formatted_address', 'geometry/location', 'type',
  ]

  try:
    # Chama a função place() para obter detalhes
    details = maps_client.place(place_id=place_id, language='pt-BR', fields=fields)
    result = details.get('result', {})

    if result:
      geometry = result.get('geometry', {})
      latitude = geometry.get('location', {}).get('lat', None)
      longitude = geometry.get('location', {}).get('lng', None)

      return {
        'Query_Original': query_original,
        'Nome': result.get('name', 'N/A'),
        'Place_ID': result.get('place_id', 'N/A'),
        'Endereço': result.get('formatted_address', 'N/A'),
        'Latitude': latitude,
        'Longitude': longitude,
        'Tipos': result.get('types', [])
      }

  except (googlemaps.exceptions.ApiError, googlemaps.exceptions.Timeout) as e:
      print(f"Erro ao recuperar detalhes para Place ID '{place_id}' (Query: '{query_original}'): {e}")

  return None

In [14]:
def extract_data_from_json(json_path):
  """
  Lê um arquivo JSON de matrícula de imóvel e extrai o nome e endereço.
  """
  with open(json_path, 'r', encoding='utf-8') as f:
    data = json.load(f)

  nome_imovel = data.get('imovel', {}).get('tipo')
  endereco_imovel = data.get('imovel', {}).get('endereco')
  matricula = data.get('matricula')

  if nome_imovel and endereco_imovel:
    return {
        'Nome': nome_imovel,
        'Endereço': endereco_imovel,
        'Matrícula': matricula
    }
  return None

# Caminho para a pasta do resultados
pasta_name = 'resultados_json'
diretorio_atual = pathlib.Path.cwd()
parent_dir = diretorio_atual.parent 
JSON_DIR = parent_dir / pasta_name

list_residencias = []

for filename in os.listdir(JSON_DIR):
    if filename.endswith('.json'):
        file_path = os.path.join(JSON_DIR, filename)
        data = extract_data_from_json(file_path)
        if data:
            list_residencias.append(data)

df_residencial = pd.DataFrame(list_residencias)

if df_residencial.empty:
    print("AVISO: Nenhum dado de residência foi carregado dos arquivos JSON. O script não continuará.")
else:
    print(f"Dados de {len(df_residencial)} imóveis carregados com sucesso!")
    print(df_residencial.head())

Dados de 5 imóveis carregados com sucesso!
                                                Nome  \
0  UNIDADE sob n.º 601, do Condomínio Residencial...   
1  APARTAMENTO sob n.º 405, do Edifício Residenci...   
2  UNIDADE AUTÔNOMA DE Nº 3102 DA TORRE B, EDIFÍC...   
3  APARTAMENTO n.º 1802, do EDIFICIO MANOÁ RESIDENCE   
4  UNIDADE AUTONOMA RESTAURANTE n.º 506, do Edifí...   

                                            Endereço Matrícula  
0  Rua Major Salustiano Ribeiro, n.º 125, no bair...    61.080  
1  Rua Tertuliano de Castro, n.º 101, esquina com...    56.068  
2  Rua Iracema Guedes Lins, no bairro Altiplano C...   115.751  
3  Rua Bacharel José de Oliveira Curchatuz, nº 52...    87.950  
4  Avenida João Mauricio, nº 581, esquina com a A...    85.648  


In [40]:
df_place_details = []

for index, row in df_residencial.iterrows():
  # 1. Cria a query de busca
  query = f'{row["Nome"]} {row["Endereço"]}'
  print(f"Buscando ID para: {query}")

  # 2. Obtém o place_id (chamada Place Search)
  place_id = get_place_id(query, maps_client)

  if place_id:
      print(f"-> Place ID encontrado: {place_id}. Buscando detalhes...")

      # 3. Obtém os detalhes completos (chamada Place Details)
      details = get_place_details(place_id, query, maps_client)

      if details:
          details['Matrícula'] = row['Matrícula']  # PRESVERVA A MATRÍCULA
          df_place_details.append(details)
      else:
          # Adiciona registro vazio se não conseguir os detalhes
            df_place_details.append({'Query_Original': query, 'Nome': row['Nome'], 'Endereço': row['Endereço'], 'Matrícula': row['Matrícula']})
  else:
      # Adiciona registro vazio se não encontrar o place_id
      df_place_details.append({'Query_Original': query, 'Nome': row['Nome'], 'Endereço': row['Endereço'], 'Matrícula': row['Matrícula']})

  # Pausa para respeitar os limites da API e evitar sobrecarga
  time.sleep(1)

# 4. Finaliza e salva
df_place_details_final = pd.DataFrame(df_place_details)

print("\nShape final do DataFrame:", df_place_details_final.shape)
df_place_details_final.to_csv('../data/place_details_complementar.csv', index=False)

Buscando ID para: UNIDADE sob n.º 601, do Condomínio Residencial EDIFÍCIO SÂNDALOS Rua Major Salustiano Ribeiro, n.º 125, no bairro de Tambauzinho
-> Place ID encontrado: ChIJoYndKkrdrAcRJGwe6MO-XhE. Buscando detalhes...
Buscando ID para: APARTAMENTO sob n.º 405, do Edifício Residencial D´UOMO DI MILANO Rua Tertuliano de Castro, n.º 101, esquina com a Rua Mirocem Cunha Lima, no bairro do Bessa
-> Place ID encontrado: ChIJow8PzZXdrAcRvWSIFlnh8w4. Buscando detalhes...
Buscando ID para: UNIDADE AUTÔNOMA DE Nº 3102 DA TORRE B, EDIFÍCIO NEXT TOWERS Rua Iracema Guedes Lins, no bairro Altiplano Cabo Branco
-> Place ID encontrado: ChIJEeyrfcvDrAcRA-xZkpX6szM. Buscando detalhes...
Buscando ID para: APARTAMENTO n.º 1802, do EDIFICIO MANOÁ RESIDENCE Rua Bacharel José de Oliveira Curchatuz, nº 527, esquina com a Rua Vanja Viana Sales, no bairro Aeroclube
-> Place ID encontrado: ChIJGypPdqLdrAcR_7qG30c8ecI. Buscando detalhes...
Buscando ID para: UNIDADE AUTONOMA RESTAURANTE n.º 506, do Edifício MAN

# Passo 02 - Informações dos estabelecimentos próximas das residências

In [42]:
BUSCAR_ESTABELECIMENTOS = [
  # Saúde
  {'keyword': 'farmácia', 'type': 'pharmacy', 'category': 'Saúde'},
  {'keyword': 'hospital ou clínica', 'type': 'hospital', 'category': 'Saúde'},

  # Educação
  {'keyword': 'escola', 'type': 'school', 'category': 'Educação'},
  {'keyword': 'universidade', 'type': 'university', 'category': 'Educação'},

  # Comércio/Serviços
  {'keyword': 'supermercado', 'type': 'supermarket', 'category': 'Comércio'},
  {'keyword': 'posto de gasolina', 'type': 'gas_station', 'category': 'Serviços'},
  {'keyword': 'academia', 'type': 'gym', 'category': 'Lazer/Fitness'},
  {'keyword': 'restaurante', 'type': 'restaurant', 'category': 'Alimentação'}
]

# Distância (raio) em metros
DISTANCIA_RAIOS = 400 # 400m

In [43]:
def get_places_nearby(location, search_keyword, distance, search_type, maps_client):
  """
  Busca locais próximos a uma coordenada usando a API Places Nearby,
  tratando a paginação.
  """
  list_places = []

  response = maps_client.places_nearby(
      location=location,
      keyword=search_keyword,
      radius=distance,
      type=search_type
  )

  list_places.extend(response.get('results', []))

  next_page_token = response.get('next_page_token')

  while next_page_token:
      time.sleep(2)
      response = maps_client.places_nearby(
          location=location,
          radius=distance,
          keyword=search_keyword,
          type=search_type,
          page_token=next_page_token
      )
      list_places.extend(response.get('results', []))
      next_page_token = response.get('next_page_token')

  return pd.DataFrame(list_places) if list_places else pd.DataFrame()

In [44]:
df_places_nearby_results = []
total_rows = len(df_place_details_final)
print(f"Iniciando busca de estabelecimentos para {total_rows} residências...")

# 1. Loop sobre cada residência (linha do DataFrame)
for idx, row in df_place_details_final.iterrows():

  residence_name = row['Nome']
  location = [row['Latitude'], row['Longitude']]
  print(f"[{idx+1}/{total_rows}] Processando: {residence_name}...")

  # 2. Loop sobre cada tipo de busca (escola, farmácia, etc.)
  for estabelecimento in BUSCAR_ESTABELECIMENTOS:

      keyword = estabelecimento['keyword']
      search_type = estabelecimento['type']
      category = estabelecimento['category']

      try:
          # 3. Chama a função para buscar locais próximos
          places_df = get_places_nearby(
              location=location,
              search_keyword=keyword,
              distance=DISTANCIA_RAIOS,
              search_type=search_type,
              maps_client=maps_client
          )

          if not places_df.empty:
              # 4. Adiciona metadados da busca e da residência
              places_df['Residência Parceira'] = residence_name
              places_df['Categoria'] = category
              places_df['Busca_Tipo'] = search_type
              places_df['Busca_Keyword'] = keyword

              df_places_nearby_results.append(places_df)

      except Exception as e:
          print(f"Erro ao buscar {category} para {residence_name}: {e}")

      time.sleep(0.5)

print("Busca concluída. Consolidando dados...")

# 5. Consolidação e Limpeza
if df_places_nearby_results:
    df_places_nearby_consolidated = pd.concat(df_places_nearby_results, ignore_index=True)

    # Extrai Lat/Lng da coluna 'geometry'
    df_places_nearby_consolidated['Latitude_Estabelecimento'] = df_places_nearby_consolidated['geometry'].map(lambda x: x['location']['lat'])
    df_places_nearby_consolidated['Longitude_Estabelecimento'] = df_places_nearby_consolidated['geometry'].map(lambda x: x['location']['lng'])

    # Seleção e Renomeação de Colunas
    df_places_nearby_final = df_places_nearby_consolidated.rename(columns={
        'name': 'Nome_Estabelecimento',
        'place_id': 'Place_ID_Estabelecimento',
        'vicinity': 'Endereço_Estabelecimento',
        'types': 'Tipos_API'
    })[['Nome_Estabelecimento', 'Place_ID_Estabelecimento', 'Endereço_Estabelecimento',
        'Residência Parceira', 'Categoria', 'Busca_Tipo', 'Busca_Keyword',
        'Latitude_Estabelecimento', 'Longitude_Estabelecimento', 'Tipos_API']]

    # Remoção de Duplicatas (se o mesmo local aparecer em duas buscas ou duas residências)
    # Mantemos a duplicação se for para residências diferentes.
    df_places_nearby_final.drop_duplicates(subset=['Place_ID_Estabelecimento', 'Residência Parceira'], keep='first', inplace=True)
    df_places_nearby_final.reset_index(drop=True, inplace=True)

    print(f"\nDataFrame final criado. Shape: {df_places_nearby_final.shape}")

    # Salvar o resultado
    df_places_nearby_final.to_csv('../data/places_nearby_complementar.csv', index=False)

else:
    print("Nenhum estabelecimento encontrado.")

Iniciando busca de estabelecimentos para 5 residências...
[1/5] Processando: Residencial Sandalos...
[2/5] Processando: Condomínio Residencial St. Tropez Residence...
[3/5] Processando: Edifício Next Towers...
[4/5] Processando: Manoá Residence...
[5/5] Processando: Manaíra Palace Residence...
Busca concluída. Consolidando dados...

DataFrame final criado. Shape: (178, 10)


# Passo 03 - Visualização da residência e dos estabelecimentos num mapa

In [46]:
def sanitize_filename(name):
    """
    Limpa o nome da residência para uso como nome de arquivo, removendo
    caracteres especiais e limitando o tamanho.
    """
    s = re.sub(r'[^\w\s-]', '', name).strip().lower()
    s = re.sub(r'[-\s]+', '_', s)
    return s[:60]


# 1. CONFIGURAÇÃO DE CORES POR CATEGORIA
# Define um dicionário de cores para as categorias de estabelecimentos próximos
COLOR_MAP = {
    'Saúde': 'blue',
    'Educação': 'green',
    'Comércio': 'orange',
    'Serviços': 'purple',
    'Lazer/Fitness': 'pink',
    'Alimentação': 'darkred',
    'Outros': 'gray'
}

print("Iniciando a geração de mapas individuais...")

if df_place_details_final.empty:
    print("ERRO: O DataFrame 'df_place_details_final' está vazio. Não é possível gerar mapas.")
else:
    MAP_OUTPUT_DIR = "../mapas_individuais"
    if not os.path.exists(MAP_OUTPUT_DIR):
        os.makedirs(MAP_OUTPUT_DIR)
    
    total_residences = len(df_place_details_final)
    
    # Loop sobre cada residência para gerar um mapa individual
    for idx, residence_row in df_place_details_final.iterrows():

        residence_name = residence_row['Nome']
        residence_lat = residence_row['Latitude']
        residence_lon = residence_row['Longitude']
        residence_mat = residence_row['Matrícula']
        
        # Garante que temos coordenadas válidas
        if pd.isna(residence_lat) or pd.isna(residence_lon):
             print(f"[{idx+1}/{total_residences}] Pulando '{residence_name}': Localização não encontrada.")
             continue

        print(f"[{idx+1}/{total_residences}] Gerando mapa para: {residence_name}...")

        # A. INICIALIZAÇÃO DO NOVO MAPA centrado na residência atual
        mapa = folium.Map(location=[residence_lat, residence_lon], zoom_start=14) # Zoom aumentado para vizinhança

        # B. ADICIONA MARCADOR DA RESIDÊNCIA (PONTO BASE)
        folium.Marker(
            location=[residence_lat, residence_lon],
            popup=f"RESIDÊNCIA: <b>{residence_name}</b><br>Endereço: {residence_row['Endereço']}",
            # Cor vermelha e ícone de casa para destaque
            icon=folium.Icon(color="red", icon="home", prefix='fa')
        ).add_to(mapa)

        # C. FILTRA os estabelecimentos próximos (Passo 02) para a residência atual
        df_nearby_single = df_places_nearby_final[
            df_places_nearby_final['Residência Parceira'] == residence_name
        ]
        
        # D. ADICIONA MARCADORES DOS ESTABELECIMENTOS PRÓXIMOS
        
        # 1. Cria FeatureGroup separado para cada categoria no mapa atual
        groups = {}
        # Assume-se que BUSCAR_ESTABELECIMENTOS está carregado do Passo 02
        for estabelecimento in BUSCAR_ESTABELECIMENTOS: 
            cat_name = estabelecimento['category']
            groups[cat_name] = folium.FeatureGroup(name=f"Próximos: {cat_name}").add_to(mapa)
            
        
        for _, nearby_row in df_nearby_single.iterrows():
            # Obtém a cor com base na categoria
            color = COLOR_MAP.get(nearby_row['Categoria'], 'gray')

            # Prepara o conteúdo do popup
            popup_html = f"""
            <b>{nearby_row['Nome_Estabelecimento']}</b><br>
            Tipo: {nearby_row['Categoria']} ({nearby_row['Busca_Tipo']})<br>
            Endereço: {nearby_row['Endereço_Estabelecimento']}<br>
            """
            
            # Adiciona o marcador ao FeatureGroup da sua respectiva categoria
            folium.Marker(
                location=[nearby_row['Latitude_Estabelecimento'], nearby_row['Longitude_Estabelecimento']],
                popup=folium.Popup(popup_html, max_width=300),
                icon=folium.Icon(color=color, icon='info-sign', prefix='fa')
            ).add_to(groups[nearby_row['Categoria']])

        # E. ADICIONA CAMADAS DE CONTROLE
        folium.LayerControl().add_to(mapa)

        # F. SALVAR O MAPA com um nome de arquivo único
        filename_base = sanitize_filename(residence_mat)
        mapa_filename = os.path.join(MAP_OUTPUT_DIR, f"mapa_{filename_base}.html")
        mapa.save(mapa_filename)
        
        print(f"-> Mapa salvo em: {mapa_filename}")

print("\nConcluído: Todos os mapas individuais foram gerados na pasta 'mapas_individuais'!")

Iniciando a geração de mapas individuais...
[1/5] Gerando mapa para: Residencial Sandalos...
-> Mapa salvo em: ../mapas_individuais\mapa_61080.html
[2/5] Gerando mapa para: Condomínio Residencial St. Tropez Residence...
-> Mapa salvo em: ../mapas_individuais\mapa_56068.html
[3/5] Gerando mapa para: Edifício Next Towers...
-> Mapa salvo em: ../mapas_individuais\mapa_115751.html
[4/5] Gerando mapa para: Manoá Residence...
-> Mapa salvo em: ../mapas_individuais\mapa_87950.html
[5/5] Gerando mapa para: Manaíra Palace Residence...
-> Mapa salvo em: ../mapas_individuais\mapa_85648.html

Concluído: Todos os mapas individuais foram gerados na pasta 'mapas_individuais'!
